# Matrix Factorization

https://towardsdatascience.com/recsys-series-part-4-the-7-variants-of-matrix-factorization-for-collaborative-filtering-368754e4fab5

## 1 - Vanilla Matrix Factorization

$R_{ui} = p_u * q_i$

Where
- $R$ is the rating matrix
- subscript $u$ refers to users
- subscript $i$ refers to items

In [11]:
import numpy as np

R = np.array(
    [[5, 3, 0, 1], [4, 0, 0, 1], [1, 1, 0, 5], [1, 0, 0, 4], [0, 1, 5, 4]], dtype=float
)
R

array([[5., 3., 0., 1.],
       [4., 0., 0., 1.],
       [1., 1., 0., 5.],
       [1., 0., 0., 4.],
       [0., 1., 5., 4.]])

In [33]:
import torch
from torch.autograd import Variable


class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)


def train(model, epochs=10, lr=0.01):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    loss_func = torch.nn.MSELoss()

    for epoch in range(epochs):
        u, i = R.nonzero()
        r = R[R.nonzero()]
        users = Variable(torch.LongTensor(u))  # replace df with your dataframe
        items = Variable(torch.LongTensor(i))  # replace df with your dataframe
        ratings = Variable(torch.FloatTensor(r))  # replace df with your dataframe

        optimizer.zero_grad()
        predictions = model(users, items)
        loss = loss_func(predictions, ratings)
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print("Epoch: {}, Loss: {}".format(epoch, loss.data.item()))


n_users, n_items = R.shape
model = MatrixFactorization(n_users, n_items, n_factors=3)
train(model, epochs=500)

Epoch: 0, Loss: 10.032804489135742
Epoch: 100, Loss: 2.9770305156707764
Epoch: 200, Loss: 0.5948803424835205
Epoch: 300, Loss: 0.22918803989887238
Epoch: 400, Loss: 0.1396285593509674


In [34]:
n_users, n_items = R.shape
r = np.zeros((n_users, n_items))
for i in range(n_users):
    for j in range(n_items):
        r[i, j] = model(torch.LongTensor([i]), torch.LongTensor([j])).item()
np.round(r, 2)

array([[5.12, 2.48, 3.75, 1.03],
       [3.97, 1.5 , 2.32, 0.96],
       [1.26, 0.44, 3.54, 4.89],
       [0.86, 0.45, 3.1 , 4.06],
       [3.74, 1.77, 4.76, 4.09]])

In [35]:
R

array([[5., 3., 0., 1.],
       [4., 0., 0., 1.],
       [1., 1., 0., 5.],
       [1., 0., 0., 4.],
       [0., 1., 5., 4.]])

## 2 - Matrix Factorization with Bias


$R_{ui} = p_u * q_i + b + b_i + b_u$

Where
- $R$ is the rating matrix
- subscript $u$ refers to users
- subscript $i$ refers to items
- $b$ is the average rating
- $b_u$ is the user bias
- $b_i$ is the item bias

In [38]:
import torch
from torch.autograd import Variable


class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_biases = torch.nn.Embedding(n_users, 1)
        self.item_biases = torch.nn.Embedding(n_items, 1)
        self.global_bias = torch.nn.Parameter(torch.zeros(1))

    def forward(self, user, item):
        pred = self.user_biases(user) + self.item_biases(item) + self.global_bias
        pred += (self.user_factors(user) * self.item_factors(item)).sum(1, keepdim=True)
        return pred.squeeze()


def train(model, epochs=10, lr=0.01):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    loss_func = torch.nn.MSELoss()

    for epoch in range(epochs):
        u, i = R.nonzero()
        r = R[R.nonzero()]
        users = Variable(torch.LongTensor(u))  # replace df with your dataframe
        items = Variable(torch.LongTensor(i))  # replace df with your dataframe
        ratings = Variable(torch.FloatTensor(r))  # replace df with your dataframe

        optimizer.zero_grad()
        predictions = model(users, items)
        loss = loss_func(predictions, ratings)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print("Epoch: {}, Loss: {}".format(epoch, loss.item()))


n_users, n_items = R.shape
model = MatrixFactorization(n_users, n_items, n_factors=3)
train(model, epochs=500)

Epoch: 0, Loss: 18.777503967285156
Epoch: 100, Loss: 1.3047934770584106
Epoch: 200, Loss: 0.2943360507488251
Epoch: 300, Loss: 0.12319314479827881
Epoch: 400, Loss: 0.06013035029172897


In [39]:
n_users, n_items = R.shape
r = np.zeros((n_users, n_items))
for i in range(n_users):
    for j in range(n_items):
        r[i, j] = model(torch.LongTensor([i]), torch.LongTensor([j])).item()
np.round(r, 2)

array([[ 5.11,  2.92,  0.93,  1.22],
       [ 3.89,  3.14,  2.38,  0.78],
       [ 0.84,  1.12,  3.51,  4.67],
       [ 1.15,  3.51,  5.53,  4.26],
       [-1.83,  0.93,  5.02,  4.07]])

In [40]:
R

array([[5., 3., 0., 1.],
       [4., 0., 0., 1.],
       [1., 1., 0., 5.],
       [1., 0., 0., 4.],
       [0., 1., 5., 4.]])

## 3 — Matrix Factorization with Side Features

Incorporating side features into a matrix factorization model can provide additional information that can improve the quality of the recommendations. These side features could be user or item attributes such as user age, user gender, item category, item price, etc.

Here's a simple example of how you might modify the above code to include user and item side features:

In [ ]:
import torch
from torch.autograd import Variable


class MatrixFactorization(torch.nn.Module):
    def __init__(
        self, n_users, n_items, n_user_features, n_item_features, n_factors=20
    ):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_biases = torch.nn.Embedding(n_users, 1)
        self.item_biases = torch.nn.Embedding(n_items, 1)
        self.user_feature_weights = torch.nn.Linear(n_user_features, 1)
        self.item_feature_weights = torch.nn.Linear(n_item_features, 1)
        self.global_bias = torch.nn.Parameter(torch.zeros(1))

    def forward(self, user, item, user_features, item_features):
        pred = self.user_biases(user) + self.item_biases(item) + self.global_bias
        pred += (self.user_factors(user) * self.item_factors(item)).sum(1, keepdim=True)
        pred += (
            self.user_feature_weights(user_features).squeeze()
            + self.item_feature_weights(item_features).squeeze()
        )
        return pred.squeeze()


def train(model, epochs=10, lr=0.01):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    loss_func = torch.nn.MSELoss()

    for epoch in range(epochs):
        users = Variable(
            torch.LongTensor(df["userId"].values)
        )  # replace df with your dataframe
        items = Variable(
            torch.LongTensor(df["movieId"].values)
        )  # replace df with your dataframe
        ratings = Variable(
            torch.FloatTensor(df["rating"].values)
        )  # replace df with your dataframe
        user_features = Variable(
            torch.FloatTensor(df["userFeatures"].values)
        )  # replace df with your dataframe
        item_features = Variable(
            torch.FloatTensor(df["itemFeatures"].values)
        )  # replace df with your dataframe

        optimizer.zero_grad()
        predictions = model(users, items, user_features, item_features)
        loss = loss_func(predictions, ratings)
        loss.backward()
        optimizer.step()

        print("Epoch: {}, Loss: {}".format(epoch, loss.item()))


n_users = 100  # replace with your actual value
n_items = 100  # replace with your actual value
n_user_features = 10  # replace with your actual value
n_item_features = 10  # replace with your actual value
model = MatrixFactorization(n_users, n_items, n_user_features, n_item_features)
train(model)

## 4 — Matrix Factorization with Temporal Features
Incorporating temporal features into a matrix factorization model can provide additional information that can improve the quality of the recommendations. These temporal features could be the time of the rating, the user's activity level at different times, seasonal trends, etc.

In [ ]:
import torch
from torch.autograd import Variable


class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_temporal_features, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_biases = torch.nn.Embedding(n_users, 1)
        self.item_biases = torch.nn.Embedding(n_items, 1)
        self.temporal_feature_weights = torch.nn.Linear(n_temporal_features, 1)
        self.global_bias = torch.nn.Parameter(torch.zeros(1))

    def forward(self, user, item, temporal_features):
        pred = self.user_biases(user) + self.item_biases(item) + self.global_bias
        pred += (self.user_factors(user) * self.item_factors(item)).sum(1, keepdim=True)
        pred += self.temporal_feature_weights(temporal_features).squeeze()
        return pred.squeeze()


def train(model, epochs=10, lr=0.01):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    loss_func = torch.nn.MSELoss()

    for epoch in range(epochs):
        users = Variable(
            torch.LongTensor(df["userId"].values)
        )  # replace df with your dataframe
        items = Variable(
            torch.LongTensor(df["movieId"].values)
        )  # replace df with your dataframe
        ratings = Variable(
            torch.FloatTensor(df["rating"].values)
        )  # replace df with your dataframe
        temporal_features = Variable(
            torch.FloatTensor(df["temporalFeatures"].values)
        )  # replace df with your dataframe

        optimizer.zero_grad()
        predictions = model(users, items, temporal_features)
        loss = loss_func(predictions, ratings)
        loss.backward()
        optimizer.step()

        print("Epoch: {}, Loss: {}".format(epoch, loss.item()))


n_users = 100  # replace with your actual value
n_items = 100  # replace with your actual value
n_temporal_features = 10  # replace with your actual value
model = MatrixFactorization(n_users, n_items, n_temporal_features)
train(model)

## 5 — Factorization Machines
Factorization Machines (FMs) are a general-purpose supervised learning algorithm that you can use for both regression and classification tasks. They are a good choice when dealing with high dimensional sparse datasets and can model complex interactions between features using factorized parameters.

Here's a simple implementation of Factorization Machines in PyTorch:


In [53]:
import torch


class FactorizationMachine(torch.nn.Module):
    def __init__(self, n_features, n_factors):
        super().__init__()
        self.n_features = n_features
        self.n_factors = n_factors
        self.linear = torch.nn.Linear(n_features, 1)
        self.v = torch.nn.Parameter(torch.randn(n_features, n_factors))

    def forward(self, x):
        linear_part = self.linear(x)
        t0 = (x @ self.v) ** 2
        t1 = (x ** 2) @ (self.v ** 2)
        factor_part = 0.5 * (t0 - t1).sum(1, keepdim=True)
        return linear_part + factor_part


def train(model, data, target, epochs=10, lr=0.01):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    loss_func = torch.nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()
        predictions = model(data)
        loss = loss_func(predictions, target)
        loss.backward()
        optimizer.step()

        print("Epoch: {}, Loss: {}".format(epoch, loss.item()))


n_features = 100  # replace with your actual value
n_factors = 10  # replace with your actual value
model = FactorizationMachine(n_features, n_factors)

# replace data and target with your actual data
data = torch.randn(1000, n_features)
target = torch.randn(1000, 1)
train(model, data, target)

Epoch: 0, Loss: 55264.94921875
Epoch: 1, Loss: 9931461.0
Epoch: 2, Loss: 1717193960062976.0
Epoch: 3, Loss: inf
Epoch: 4, Loss: nan
Epoch: 5, Loss: nan
Epoch: 6, Loss: nan
Epoch: 7, Loss: nan
Epoch: 8, Loss: nan
Epoch: 9, Loss: nan


## Setting data loader

In [71]:
import torch
from torch.utils.data import DataLoader, Dataset


class MovieDataset(Dataset):
    def __init__(self, users, items, ratings):
        self.users = users
        self.items = items
        self.ratings = ratings

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return {
            "user": torch.tensor(self.users[idx], dtype=torch.long),
            "item": torch.tensor(self.items[idx], dtype=torch.long),
            "rating": torch.tensor(self.ratings[idx], dtype=torch.float),
        }


# replace with your actual data
users, items = R.nonzero()
ratings = R[R.nonzero()]

dataset = MovieDataset(users, items, ratings)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

for batch in data_loader:
    print(batch)

{'user': tensor([2]), 'item': tensor([1]), 'rating': tensor([1.])}
{'user': tensor([0]), 'item': tensor([1]), 'rating': tensor([3.])}
{'user': tensor([4]), 'item': tensor([1]), 'rating': tensor([1.])}
{'user': tensor([4]), 'item': tensor([3]), 'rating': tensor([4.])}
{'user': tensor([3]), 'item': tensor([0]), 'rating': tensor([1.])}
{'user': tensor([4]), 'item': tensor([2]), 'rating': tensor([5.])}
{'user': tensor([1]), 'item': tensor([3]), 'rating': tensor([1.])}
{'user': tensor([3]), 'item': tensor([3]), 'rating': tensor([4.])}
{'user': tensor([0]), 'item': tensor([3]), 'rating': tensor([1.])}
{'user': tensor([2]), 'item': tensor([0]), 'rating': tensor([1.])}
{'user': tensor([1]), 'item': tensor([0]), 'rating': tensor([4.])}
{'user': tensor([2]), 'item': tensor([3]), 'rating': tensor([5.])}
{'user': tensor([0]), 'item': tensor([0]), 'rating': tensor([5.])}
